# Intelligent Agents: Reflex-Based Agents for the Vacuum-cleaner World


## Instructions

Total Points: Undergrads 100 / Graduate students 110

Complete this notebook. Use the provided notebook cells and insert additional code and markdown cells as needed. Submit the completely rendered notebook as a PDF file. 

## Introduction

In this assignment you will implement a simulator environment for an automatic vacuum cleaner robot, a set of different reflex-based agent programs, and perform a comparison study for cleaning a single room. Focus on the __cleaning phase__ which starts when the robot is activated and ends when the last dirty square in the room has been cleaned. Someone else will take care of the agent program needed to navigate back to the charging station after the room is clean.

## PEAS description of the cleaning phase

__Performance Measure:__ Each action costs 1 energy unit. The performance is measured as the sum of the energy units used to clean the whole room.

__Environment:__ A room with $n \times n$ squares where $n = 5$. Dirt is randomly placed on each square with probability $p = 0.2$. For simplicity, you can assume that the agent knows the size and the layout of the room (i.e., it knows $n$). To start, the agent is placed on a random square.

__Actuators:__ The agent can clean the current square (action `suck`) or move to an adjacent square by going `north`, `east`, `south`, or `west`.

__Sensors:__ Four bumper sensors, one for north, east, south, and west; a dirt sensor reporting dirt in the current square.  


## The agent program for a simple randomized agent

The agent program is a function that gets sensor information (the current percepts) as the arguments. The arguments are:

* A dictionary with boolean entries for the for bumper sensors `north`, `east`, `west`, `south`. E.g., if the agent is on the north-west corner, `bumpers` will be `{"north" : True, "east" : False, "south" : False, "west" : True}`.
* The dirt sensor produces a boolean.

The agent returns the chosen action as a string.

Here is an example implementation for the agent program of a simple randomized agent: 

In [224]:
import numpy as np

actions = ["north", "east", "west", "south", "suck"]

def simple_randomized_agent(bumpers, dirty):
    return np.random.choice(actions)

In [225]:
# define percepts (current location is NW corner and it is dirty)
bumpers = {"north" : True, "east" : False, "south" : False, "west" : True}
dirty = True

# call agent program function with percepts and it returns an action
simple_randomized_agent(bumpers, dirty)

'east'

__Note:__ This is not a rational intelligent agent. It ignores its sensors and may bump into a wall repeatedly or not clean a dirty square. You will be asked to implement rational agents below.

## Simple environment example

We implement a simple simulation environment that supplies the agent with its percepts.
The simple environment is infinite in size (bumpers are always `False`) and every square is always dirty, even if the agent cleans it. The environment function returns a performance measure which is here the number of cleaned squares (since the room is infinite and all squares are constantly dirty, the agent can never clean the whole room as required in the PEAS description above). The energy budget of the agent is specified as `max_steps`. 

In [226]:
def simple_environment(agent, max_steps, verbose = True):
    num_cleaned = 0
    
    for i in range(max_steps):
        dirty = True
        bumpers = {"north" : False, "south" : False, "west" : False, "east" : False}

        action = agent(bumpers, dirty)
        if (verbose): print("step", i , "- action:", action) 
        
        if (action == "suck"): 
            num_cleaned = num_cleaned + 1
        
    return num_cleaned
        


Do one simulation run with a simple randomized agent that has enough energy for 20 steps.

In [227]:
simple_environment(simple_randomized_agent, max_steps = 20)

step 0 - action: east
step 1 - action: east
step 2 - action: east
step 3 - action: east
step 4 - action: east
step 5 - action: east
step 6 - action: south
step 7 - action: east
step 8 - action: east
step 9 - action: south
step 10 - action: south
step 11 - action: east
step 12 - action: south
step 13 - action: suck
step 14 - action: suck
step 15 - action: west
step 16 - action: north
step 17 - action: west
step 18 - action: north
step 19 - action: west


2

# Tasks

## General [10 Points]

1. Make sure that you use the latest version of this notebook. Sync your forked repository and pull the latest revision. 
2. Your implementation can use libraries like math, numpy, scipy, but not libraries that implement inteligent agents or complete search algorithms. Try to keep the code simple! In this course, we want to learn about the algorithms and we often do not need to use object-oriented design.
3. You notebook needs to be formated professionally. 
    - Add additional markdown blocks for your description, comments in the code, add tables and use mathplotlib to produce charts where appropriate
    - Do not show debugging output or include an excessive amount of output.
    - Check that your PDF file is readable. For example, long lines are cut off in the PDF file. You don't have control over page breaks, so do not worry about these.
4. Document your code. Add a short discussion of how your implementation works and your design choices.


## Task 1: Implement a simulation environment [20 Points]

The simple environment above is not very realistic. Your environment simulator needs to follow the PEAS description from above. It needs to:

* Initialize the environment by storing the state of each square (clean/dirty) and making some dirty. ([Help with random numbers and arrays in Python](https://github.com/mhahsler/CS7320-AI/blob/master/Python_Code_Examples/random_numbers_and_arrays.ipynb))
* Keep track of the agent's position.
* Call the agent function repeatedly and provide the agent function with the sensor inputs.  
* React to the agent's actions. E.g, by removing dirt from a square or moving the agent around unless there is a wall in the way.
* Keep track of the performance measure. That is, track the agent's actions until all dirty squares are clean and count the number of actions it takes the agent to complete the task.

The easiest implementation for the environment is to hold an 2-dimensional array to represent if squares are clean or dirty and to call the agent function in a loop until all squares are clean or a predefined number of steps have been reached (i.e., the robot runs out of energy).

The simulation environment should be a function like the `simple_environment()` and needs to work with the simple randomized agent program from above. Use the same environmnt for all your agent implementations in the tasks below.

*Note on debugging:* Debugging is difficult. Make sure your environment prints enough information when you use `verbose = True`. Also, implementing a function that the environment can use to displays the room with dirt and the current position of the robot at every step is very useful.  

In [2]:
import numpy as np
import random

actions = ["north", "east", "west", "south", "suck"]


def simple_randomized_agent(bumpers, dirty):
    return np.random.choice(actions)

def simple_environment(agent, max_steps, n, verbose=True):
    num_cleaned = 0

    # -------------------- Create Environment --------------------
    num_dirty = 0
    # n = 5
    rows, cols = (n, n)
    environment = [[0 for i in range(cols)] for j in range(rows)]
    for i in range(n):
        for j in range(n):
            dirt_odds = random.randrange(1, 11)
            if dirt_odds <= 2:
                environment[i][j] = 1
                num_dirty += 1

    # -------------------- Clean Environment --------------------
    dirty = False
    bumpers = {"north": False, "south": False, "west": False, "east": False}
    x_coordinate = int(random.randrange(0, n))
    y_coordinate = int(random.randrange(0, n))
    step_num = 0
    while num_dirty != num_cleaned:
        action = agent(bumpers, dirty)
        if (verbose): print("step", step_num, "- action:", action)
        if action == "north":
            if y_coordinate != 0:
                y_coordinate -= 1

        elif action == "south":
            if y_coordinate != n-1:
                y_coordinate += 1

        elif action == "east":
            if x_coordinate != n-1:
                x_coordinate += 1

        elif action == "west":
            if x_coordinate != 0:
                x_coordinate -= 1

        elif action == "suck":
            if environment[y_coordinate][x_coordinate] == 1:
                environment[y_coordinate][x_coordinate] = 0
                num_cleaned = num_cleaned + 1
                # display_environment(environment, n)  # For DEBUG Purposes
        step_num += 1

    print(step_num)
    return step_num


simple_environment(simple_randomized_agent, max_steps=20, n=5, verbose=False)

295


295

## Task 2:  Implement a simple reflex agent [10 Points] 

The simple reflex agent randomly walks around but reacts to the bumper sensor by not bumping into the wall and to dirt with sucking. Implement the agent program as a function.

_Note:_ Agents cannot directly use variable in the environment. They only gets the percepts as the arguments to the agent function.

In [394]:
import numpy as np
import random

actions = ["north", "east", "west", "south", "suck"]

x_coordinate = 0
y_coordinate = 0
step_num = 0


def reflex_agent(bumpers, dirty):
    global x_coordinate
    global y_coordinate
    global step_num

    achievable_action = False
    if dirty is True:
        step_num += 1
        return "suck"
    else:
        while achievable_action is False:
            action = np.random.choice(actions)
            if action == "north" and bumpers["north"] is True:
                achievable_action = False
            elif action == "south" and bumpers["south"] is True:
                achievable_action = False
            elif action == "east" and bumpers["east"] is True:
                achievable_action = False
            elif action == "west" and bumpers["west"] is True:
                achievable_action = False
            elif action == "suck":
                achievable_action = False
            else:
                achievable_action = True
        step_num += 1
        return action


def simple_environment2(agent, max_steps, n, verbose=True):
    num_cleaned = 0
    global x_coordinate
    global y_coordinate
    global step_num
    step_num = 0

    # -------------------- Create Environment --------------------
    num_dirty = 0
    rows, cols = (n, n)
    environment = [[0 for i in range(cols)] for j in range(rows)]
    for i in range(n):
        for j in range(n):
            dirt_odds = random.randrange(1, 11)
            if dirt_odds <= 2:
                environment[i][j] = 1
                num_dirty += 1

    # -------------------- Clean Environment --------------------
    dirty = False
    bumpers = {"north": False, "south": False, "west": False, "east": False}
    x_coordinate = int(random.randrange(0, n))
    y_coordinate = int(random.randrange(0, n))
    if y_coordinate == 0:
        bumpers["north"] = True
    if y_coordinate == n-1:
        bumpers["south"] = True
    if x_coordinate == 0:
        bumpers["west"] = True
    if x_coordinate == n-1:
        bumpers["east"] = True

    if environment[y_coordinate][x_coordinate] == 1:
        dirty = True

    while num_dirty != num_cleaned:
        action = agent(bumpers, dirty)
        if action == "north":
            y_coordinate -= 1
        elif action == "south":
            y_coordinate += 1
        elif action == "east":
            x_coordinate += 1
        elif action == "west":
            x_coordinate -= 1
        elif action == "suck":
            environment[y_coordinate][x_coordinate] = 0
            num_cleaned = num_cleaned + 1
            dirty = False

        if (verbose): print("step", step_num, "- action:", action)
        bumpers = {"north": False, "south": False, "west": False, "east": False}
        if y_coordinate == 0:
            bumpers["north"] = True
        if y_coordinate == n - 1:
            bumpers["south"] = True
        if x_coordinate == 0:
            bumpers["west"] = True
        if x_coordinate == n - 1:
            bumpers["east"] = True

        if environment[y_coordinate][x_coordinate] == 1:
            dirty = True

    print(step_num)
    return step_num


simple_environment2(reflex_agent, max_steps=20, n=5, verbose=False)


249


249

## Task 3: Implement a model-based reflex agent [20 Points]

Model-based agents use a state to keep track of what they have done and perceived so far. Your agent needs to find out where it is located and then keep track of its current location. You also need a set of rules based on the state and the percepts to make sure that the agent will clean the whole room. For example, the agent can move to a corner to determine its location and then it can navigate through the whole room and clean dirty squares.

Describe how you define the __agent state__ and how your agent works before implementing it. ([Help with implementing state information on Python](https://github.com/mhahsler/CS7320-AI/blob/master/Python_Code_Examples/store_agent_state_information.ipynb))

In [230]:
# This agent starts off by navigating north until the north bumper is triggered. 
# Then it moves west until the west bumper is triggered and it knows it is in 
# location (0,0). After that, the robot snakes down 
# the rectangular area. It snakes by heading east until it hits the east wall, 
# moving south once, heading back west until it hits the west wall, moving south once, 
# and finally heading back east again. 

In [366]:
import numpy as np
import random

actions = ["north", "east", "west", "south", "suck"]

x_coordinate = 0
y_coordinate = 0
step_num = 0
north_found = False
west_found = False
east_west = True
no_go_south = True


def model_agent(bumpers, dirty):
    global x_coordinate
    global y_coordinate
    global step_num
    global north_found
    global west_found
    global east_west
    global no_go_south

    if dirty is True:
        return "suck"

    if north_found is False:
        return "north"
    elif west_found is False:
        return "west"

    if bumpers["east"] is True and no_go_south is False:
        east_west = False
        no_go_south = True
        return "south"
    elif bumpers["west"] is True and no_go_south is False:
        east_west = True
        no_go_south = True
        return "south"
    elif east_west is True:
        no_go_south = False
        return "east"
    elif east_west is False:
        no_go_south = False
        return "west"


def display_environment(array, edge_size):  # For DEBUG purposes
    for i in range(edge_size):
        print(f"{array[i]}")


def simple_environment3(agent, max_steps, n,verbose=True):
    global x_coordinate
    global y_coordinate
    global step_num
    global north_found
    global west_found
    global east_west
    global no_go_south

    # -------------------- Create Environment --------------------
    num_dirty = 0
    step_num = 0
    num_cleaned = 0
    x_coordinate = 0
    y_coordinate = 0
    north_found = False
    west_found = False
    east_west = True
    no_go_south = True
    rows, cols = (n, n)
    environment = [[0 for i in range(cols)] for j in range(rows)]
    for i in range(n):
        for j in range(n):
            dirt_odds = random.randrange(1, 11)
            if dirt_odds <= 2:
                environment[i][j] = 1
                num_dirty += 1

    # -------------------- Clean Environment --------------------
    dirty = False
    bumpers = {"north": False, "south": False, "west": False, "east": False}
    x_coordinate = int(random.randrange(0, n))
    y_coordinate = int(random.randrange(0, n))
    if y_coordinate == 0:
        bumpers["north"] = True
        north_found = True
    if y_coordinate == n-1:
        bumpers["south"] = True
    if x_coordinate == 0:
        bumpers["west"] = True
        west_found = True
    if x_coordinate == n-1:
        bumpers["east"] = True

    while num_dirty != num_cleaned:
        action = agent(bumpers, dirty)
        step_num += 1
        if action == "north":
            y_coordinate -= 1
        elif action == "south":
            y_coordinate += 1
        elif action == "east":
            x_coordinate += 1
        elif action == "west":
            x_coordinate -= 1
        elif action == "suck":
            environment[y_coordinate][x_coordinate] = 0
            num_cleaned = num_cleaned + 1
            dirty = False

        if (verbose): print("step", step_num, "- action:", action)
        bumpers = {"north": False, "south": False, "west": False, "east": False}
        if y_coordinate == 0:
            if north_found is False:
                north_found = True
            bumpers["north"] = True
        if y_coordinate == n - 1:
            bumpers["south"] = True
        if x_coordinate == 0:
            if west_found is False:
                west_found = True
            bumpers["west"] = True
        if x_coordinate == n - 1:
            bumpers["east"] = True

        if environment[y_coordinate][x_coordinate] == 1:
            dirty = True

    print(step_num)
    return step_num



simple_environment3(model_agent, max_steps=20, n=5, verbose=False)


30


30

## Task 4: Simulation study [30 Points]

Compare the performance (the performance measure is defined in the PEAS description above) of the agents using  environments of different size. E.g., $5 \times 5$, $10 \times 10$ and
$100 \times 100$. Use 100 random runs for each. Present the results using tables and graphs. Discuss the differences between the agents. 
([Help with charts and tables in Python](https://github.com/mhahsler/CS7320-AI/blob/master/Python_Code_Examples/charts_and_tables.ipynb))

In [3]:
for i in range(100):
    simple_environment(simple_randomized_agent, max_steps=20, n=100, verbose=False)
for j in range(100):
    simple_environment2(reflex_agent, max_steps=20, n=100, verbose=False)
for z in range(100):
    simple_environment3(model_agent, max_steps=20, n=100, verbose=False)

Fill out the following table with the average performance measure for 100 random runs (you may also create this table with code):

| Size     | Randomized Agent | Simple Reflex Agent | Model-based Reflex Agent |
|----------|------------------|---------------------|--------------------------|
| 5x5     | 465| 104| 12094|
| 10x10   | 2987| 903| 124|
| 100x100 | 839218| 336621| 28|

Add charts to compare the performance of the different agents.

|Simple|Simple|Simple|Reflex|Reflex|Reflex|Model Based|Model Based|Model Based|
|------|------|------|------|------|------|-----------|-----------|-----------|
|Size|5X5|10x10|100x100|5X5|10x10|100x100|5X5|10x10|100x100|
|Averages|464.96|2986.29|839217.29|103.92|902.95|336620.2|27.74|123.67|12093.19|
|Test 1|127|3429|774456|49|1062|251743|17|118|12126|
|Test 2|1334|3080|888668|80|607|331033|33|122|12186|
|Test 3|302|3843|830312|81|1371|341793|31|117|12091|
|Test 4|260|2396|896319|144|657|348527|40|127|12085|
|Test 5|412|6860|843596|69|741|491554|27|113|12122|
|Test 6|466|4036|784747|93|944|309957|27|119|12046|
|Test 7|1009|2923|623499|79|1070|318919|30|100|12119|
|Test 8|627|1445|806152|173|681|252490|13|108|12104|
|Test 9|128|1845|711923|68|1742|264052|25|129|12056|
|Test 10|387|2453|652524|221|1926|548023|24|129|12128|
|Test 11|391|2236|758858|39|848|243103|29|120|12046|
|Test 12|234|3213|627255|97|2049|252889|28|121|12039|
|Test 13|99|2764|960608|225|649|299542|24|130|12119|
|Test 14|469|2302|1189480|49|662|305330|0|129|12091|
|Test 15|171|2263|739892|101|762|272656|19|126|12038|
|Test 16|934|3847|829051|109|1789|331433|30|134|12144|
|Test 17|959|5395|835892|130|1697|380270|32|127|12195|
|Test 18|240|4575|851995|77|607|355643|30|138|12100|
|Test 19|572|2139|1015576|24|856|357291|35|123|12041|
|Test 20|168|2021|677014|119|1343|513333|26|124|12040|
|Test 21|764|3862|650428|48|371|227977|31|111|12064|
|Test 22|398|2439|821146|142|1300|419562|31|135|12127|
|Test 23|429|2683|622661|193|875|233892|28|129|12017|
|Test 24|318|2808|1214075|59|582|452919|28|123|11998|
|Test 25|428|1340|1130192|145|337|407062|24|122|12133|
|Test 26|410|3222|867479|127|1134|370383|34|130|12112|
|Test 27|377|5065|650112|37|405|295162|36|121|12105|
|Test 28|279|2155|955500|169|1370|253103|32|124|12081|
|Test 29|294|5247|1494359|177|1055|349604|22|127|12070|
|Test 30|554|4068|800244|62|651|260160|35|126|12077|
|Test 31|393|2887|1117501|34|972|246452|28|121|12111|
|Test 32|319|2612|663141|155|1200|279459|30|103|12094|
|Test 33|72|3422|913893|21|1440|456307|14|123|12121|
|Test 34|311|1810|920685|70|642|290936|34|102|12121|
|Test 35|260|5765|694551|72|377|260756|17|108|12070|
|Test 36|302|3044|734248|110|1236|302569|33|129|12065|
|Test 37|752|2161|853003|61|1586|354984|34|108|12023|
|Test 38|263|2309|712224|96|2224|243695|20|131|12188|
|Test 39|360|2993|600282|249|556|421461|27|131|12076|
|Test 40|487|3477|885835|88|2008|578719|12|117|12149|
|Test 41|390|4114|934286|89|829|211793|33|131|12074|
|Test 42|119|2321|813889|387|848|362486|33|119|12127|
|Test 43|502|2670|707393|144|503|502562|12|130|12040|
|Test 44|538|1880|866668|375|567|247652|27|130|12162|
|Test 45|337|4542|942314|101|659|289051|27|129|12095|
|Test 46|984|2407|737154|342|1129|294758|20|126|11997|
|Test 47|183|3566|683668|93|637|350484|34|121|12142|
|Test 48|724|2077|1385082|0|628|258647|32|126|12115|
|Test 49|606|1277|958904|50|818|286195|21|132|12031|
|Test 50|332|2706|872981|85|502|289146|29|128|11968|
|Test 51|341|7210|760847|32|432|298691|31|131|12106|
|Test 52|132|1945|831757|52|561|321397|30|115|12122|
|Test 53|640|3346|729948|165|675|256301|18|136|12071|
|Test 54|414|2383|878800|13|708|319271|40|141|12012|
|Test 55|267|1301|994023|50|843|362546|27|130|12057|
|Test 56|317|3848|849424|353|1314|578423|35|126|12088|
|Test 57|862|3245|845804|10|682|301428|25|133|12227|
|Test 58|114|2119|745607|29|998|292064|22|123|12114|
|Test 59|136|2570|786189|59|1938|348289|18|119|12025|
|Test 60|752|5236|965160|106|590|395381|32|127|12104|
|Test 61|545|2763|947874|104|503|609707|29|129|11984|
|Test 62|191|4687|793411|79|387|386095|33|120|12096|
|Test 63|623|1733|773088|207|555|292727|31|127|12071|
|Test 64|213|5481|762093|91|666|306427|23|135|12133|
|Test 65|413|3033|779146|76|2443|335563|17|131|12190|
|Test 66|352|3362|805371|66|875|355906|30|120|12038|
|Test 67|316|2603|1054549|116|921|388522|34|122|12116|
|Test 68|765|2985|696337|103|328|324124|31|113|12109|
|Test 69|904|2226|903847|31|1278|338509|33|128|12039|
|Test 70|1356|4945|791473|56|511|236673|27|119|12120|
|Test 71|250|3681|789145|37|882|529158|32|128|12055|
|Test 72|725|4629|710496|235|1274|456279|20|127|12062|
|Test 73|1034|2505|917790|179|555|410764|34|115|12026|
|Test 74|530|1851|779255|68|898|237186|30|124|12227|
|Test 75|474|3206|992742|85|788|260262|31|123|12093|
|Test 76|582|2583|668270|23|641|267940|34|126|12090|
|Test 77|457|2430|1250280|206|739|375981|29|134|12098|
|Test 78|606|1951|670415|95|386|351004|34|108|12082|
|Test 79|519|1694|778871|117|836|338613|32|120|12066|
|Test 80|79|2030|867809|49|1061|275049|24|136|12093|
|Test 81|199|2023|771854|101|1450|555759|29|119|11975|
|Test 82|463|2819|777283|27|848|326574|22|132|12157|
|Test 83|413|3096|749428|10|427|339294|8|119|12068|
|Test 84|397|1907|895150|38|629|350533|30|133|12101|
|Test 85|609|1868|962028|62|688|272080|33|127|12107|
|Test 86|452|2821|658580|95|1244|261365|33|130|12037|
|Test 87|117|2622|1024187|125|617|254430|20|121|12038|
|Test 88|212|2335|1186498|170|1105|267280|33|119|12098|
|Test 89|727|3974|762419|63|993|256494|22|124|12188|
|Test 90|709|2580|594948|78|973|260142|31|133|12084|
|Test 91|595|1958|898987|346|527|269135|28|95|12198|
|Test 92|176|2292|672352|62|969|267675|28|125|12129|
|Test 93|939|899|913084|92|954|296711|34|109|12095|
|Test 94|577|3979|711620|100|686|239507|24|128|12096|
|Test 95|348|2224|774158|56|637|226400|29|113|12154|
|Test 96|441|2935|732109|69|646|311233|35|118|12058|
|Test 97|268|2965|847394|11|573|518991|33|124|12118|
|Test 98|457|1903|820951|68|570|590672|31|124|12143|
|Test 99|736|3080|787866|89|547|379098|35|134|12140|
|Test 100|978|2774|759297|30|440|300850|32|127|12152|


In [ ]:
# Your graphs and discussion of the results goes here

## Task 5: Robustness of the agent implementations [10 Points] 

Describe how your agent implementations will perform 

* if it is put into a rectangular room with unknown size, 
* if the cleaning area can have an iregular shape (e.g., a hallway connecting two rooms), or 
* if the room contains obstacles (i.e., squares that it cannot pass through and trigger the bumper sensors).

In [ ]:
# if it is put into a rectangular room with unknown size,
#   Answer: If the agent was put into a rectangular room with an unknown size 
# with dimensions (l,w) the agent would function properly. In any rectangular 
# room the agent would go north until the north bumper 
# is triggered and then it would go left until the west bumper was triggered. 
# This would set the agent at coordinats (0,0). The agent would then continue 
# to move east until the east bumper was triggered, move south
# once and then continue back west. Once the west bumper is triggered it would 
# move south once, then east, and continue the cycle down the rectangle until 
# all dirty tiles were cleaned. Ff the cleaning area can have an iregular shape 
# (e.g., a hallway connecting two rooms), or
#   Answer: If there was an irregular environment shape the agent would be 
# able to clean parts of the area, but not the entire area. Using the hallway 
# example from above. I would be able to fully clean one of the 3
# rectangular areas (Room 1, Hallway, Room 2), but I would not be able to 
# intentionally move from one room to another. This is because the agent would 
# "snake" through one of the 3 rooms and eventually get to an
# corner and get stuck bumping into a wall (Most likely the south wall).
# if the room contains obstacles (i.e., squares that it cannot pass through 
# and trigger the bumper sensors).
#   Answer: The agent does not have the inteligence to work around obsticles 
# in the room. If the obsticle triggered the North bumper when the agent was 
# initially trying to navigate to the north wall then no dirty 
# spots to the north of the obsticle would be cleaned. If the obsticle triggered
# the West bumper when the agent was initially trying to navigate to the (0,0) 
# coordinate then no dirty spots to the west of the 
# obsticle would be cleaned. If the agent was heading east after the agent found 
# the (0,0) coordinate and the obsticle triggered it's east bumper, it would move 
# south and head back west. This would mean that any 
# dirty spots directly east (in the same row) of the obsticle would remain uncleaned. 
# If the agent was snaking and heading west and the obsticle triggered it's west bumper, 
# it would move south and head back east. This would mean that any dirty spots directly 
# west (in the same row) of the obsticle would remain uncleaned. If the obsticle triggered 
# the south bumper at any time the agent would act as if the obsticle wasn't 
# there and attempt to perform the next action using it's other sensors (most likely 
# eventually breaking in the process). 

## Graduate student advanced task: Obstacles [10 Points]

__Undergraduate students:__ This is a bonus task you can attempt if you like [+5 Bonus Points].

1. Change your simulation environment tor run experiments for the following problem: Add random obstacle squares that also trigger the bumper sensor. The agent does not know where the obstacles are. Observe how this changes the performance of the three implementations.

2. Describe what would need to be done to perform better with obstacles. Add code if you can. 

In [ ]:
# N/A

## More advanced implementation tasks

* __Agent for and environment with obstacles:__ Implement an agent for an environment where the agent does not know how large the environment is (we assume it is rectangular), where it starts or where the obstacles are. An option would be to always move to the closest unchecked/uncleaned square (note that this is actualy depth-first search).

* __Utility-based agent:__ Change the environment for a $5 \times 5$ room, so each square has a fixed probability of getting dirty again. For the implementation, we give the environment a 2-dimensional array of probabilities. The utility of a state is defined as the number of currebntly clean squares in the room. Implement a utility-based agent that maximizes the expected utility over one full charge which lasts for 100000 time steps. To do this, the agent needs to learn the probabilities with which different squares get dirty again. This is very tricky!

In [ ]:
# Your ideas/code